# ETL para dataset especifico para la funcion UserForGenre

In [12]:
import pandas as pd
import pyarrow.parquet as pq

### Cargamos los Datasets

In [13]:
steam = pd.read_json('../Datasets/Steam_Games_Limpio.json.gz', compression='gzip')
review = pd.read_json('../Datasets/User_Reviews_Limpio.json.gz', compression='gzip')
items = pq.read_table('Datasets OPS/items.parquet').to_pandas()

#### Nos quedamos con la columna necesaria

In [14]:
generosAVer = steam['genres']

#### Guardamos en listaDeGeneros todos los generos que existen en el dataset

In [17]:
listaDeGeneros = []
for linea in generosAVer:
  for i in linea:
    if i not in listaDeGeneros:
      listaDeGeneros.append(i)

generos = {'genero':listaDeGeneros}
df = pd.DataFrame(generos)
df[['usuario','Horas jugadas']] = None

#### Vemos los items de todos los usuarios

In [27]:
user_items = items # o pd.read_json('API/Datasets/items.json') 
itemsDeUsuario = items.explode('items')
userID = itemsDeUsuario['user_id']
itemsDeUsuario = pd.json_normalize(itemsDeUsuario['items'])
itemsDeUsuario['user_id'] = list(userID)
itemsDeUsuario.drop(columns=['playtime_2weeks','item_name'], inplace=True)
itemsDeUsuario.dropna(subset='item_id', inplace=True)

#### Unimos los items de todos los usuarios con los que tenemos informacion en el dataset de steamgames

In [28]:
itemsDeUsuario['item_id'] = itemsDeUsuario['item_id'].astype(int)
itemsDeUsuario = itemsDeUsuario.merge(steam[['id','genres','release_date']], left_on='item_id', right_on='id')
itemsDeUsuario

,item_id,playtime_forever,user_id,id,genres,release_date
0,10,6.0,76561197970982479,10,[Action],2000
1,20,0.0,76561197970982479,20,[Action],1999
2,30,7.0,76561197970982479,30,[Action],2003
3,40,0.0,76561197970982479,40,[Action],2001
4,50,0.0,76561197970982479,50,[Action],1999
...,...,...,...,...,...,...
4201145,304930,677.0,76561198329548331,304930,"[Action, Adventure, Casual, Free to Play, Indie]",2017
4201146,227940,43.0,76561198329548331,227940,"[Action, Free to Play, Indie, Massively Multip...",2016
4201147,346330,0.0,76561198329548331,346330,"[Action, Adventure, Free to Play, Indie, Early...",2016
4201148,388490,3.0,76561198329548331,388490,"[Adventure, Free to Play]",2015


#### Calculamos por genero cual es el usuario que mas jugo a tal genero, y la cantidad de horas por año de lanzamiento del juego que tuvo

In [21]:
for i in df['genero'].values:
    genero = i

    #Juegos del genero pedido
    mascara = itemsDeUsuario['genres'].apply(lambda x: genero in x)

    #Calcular las horas que tiene un usuario con juegos del genero
    sumaDehoras = itemsDeUsuario[mascara].groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)
    
    if not sumaDehoras.empty:
        usuario = sumaDehoras.head(1).index[0]
    else:
        usuario = None

    df.loc[df['genero'] == genero, 'usuario'] = usuario

    aniosYhoras = itemsDeUsuario[(itemsDeUsuario['user_id'] == usuario) & mascara].groupby('release_date')['playtime_forever'].sum()
    anio = aniosYhoras.index
    horasJugadas = {}
    for horas, anio in enumerate(anio):
        horasJugadas[f'Año {anio}'] = f'Horas: {aniosYhoras.iloc[horas]}'

    df.loc[df['genero'] == genero, 'Horas jugadas'] = [horasJugadas]


In [8]:
df['genero'] = 'Action'

#### Hacemos unos ultimos retoques en el data set para exportarlo y poder utilizarlo en la funcion

In [29]:
df.dropna(subset=usuario, inplace=True)
df.reset_index(drop=True,inplace=True)

In [30]:
df['genero'] = df['genero'].apply(lambda x: x.capitalize())

In [31]:
df

,genero,usuario,Horas jugadas
0,Action,Sp3ctre,"{'Año 1993': 'Horas: 0.0', 'Año 1995': 'Horas:..."
1,Casual,REBAS_AS_F-T,"{'Año 1999': 'Horas: 0.0', 'Año 2002': 'Horas:..."
2,Indie,REBAS_AS_F-T,"{'Año 1999': 'Horas: 0.0', 'Año 2001': 'Horas:..."
3,Simulation,jimmynoe,"{'Año 2003': 'Horas: 10.0', 'Año 2006': 'Horas..."
4,Strategy,shinomegami,"{'Año 1993': 'Horas: 22001.0', 'Año 1995': 'Ho..."
...,...,...,...
68,Remake,76561198040188061,{'Año 2012': 'Horas: 4016.0'}
69,Time attack,76561198040188061,{'Año 2012': 'Horas: 4016.0'}
70,Documentary,2448531,{'Año 2011': 'Horas: 2572.0'}
71,Controller,76561197995146150,{'Año 2011': 'Horas: 15820.0'}


#### Exportamos el dataset para su posterior uso en la API

In [ ]:
df.to_json(".../Datasets/genre.json.gz", compression='gzip')